# 1. Preprocess Data

In this notebook we preprocess genomic data associated with DNA double-strand break (DSB) and repair experiments in human and yeast cells performed by [Jeon et al.](https://www.nature.com/articles/s41467-024-51457-9).

In these experiments, DSBs are introduced into DNA molecules of known sequence at distinct, prescribed positions (A, B, AB, or C/D), together with additional controlled modifiers. After break induction and repair, the resulting DNA sequences are collected via high-throughput sequencing.

Each experiment is defined by a combination of the following parameters:

- **Break location**: DNA cuts are introduced at specific CRISPR target sites.  
  These are labeled **A** or **B** (single cuts), **AB** (two cuts forming a gap), or **C/D** (alternative cut sites in a different construct).

- **RNA orientation**: The RNA transcript associated with the DNA can be  
  **sense** (same orientation as the DNA sequence) or **antisense** (complementary orientation).

- **Cellular background**:  
  **WT (wild type)** cells have intact repair machinery, while **KO (knock-out)** cells lack a specific RNA/DNA-processing enzyme (RNase H), altering repair dynamics.

- **RNase condition (R1 vs R2)**:  
  These labels distinguish different RNase H contexts (e.g., normal expression versus altered or deleted activity), allowing the role of RNA–DNA interactions in repair to be tested.

- **Branch**:  
  Indicates whether the RNA transcript contains an intron branch point.  
  **BranchΔ** denotes deletion of the branch site, preventing RNA splicing and modifying RNA structure.

- **CMV**:  
  Refers to the presence of a strong **CMV promoter**, which increases transcription levels.  
  Removing CMV lowers RNA production while keeping the DNA sequence unchanged.

For each combination of parameters, the experiment is performed with **four biological replicates**. 

After sequencing, the repaired DNA sequences are aligned to the original reference sequence. Reads with too many substitutions are discarded as noise. The raw data used here (stored in `data/raw`) was generated using the pipeline of [Channagiri et al.](https://github.com/tchannagiri/DSBplot), producing files named `window_freq_withoutSubst.tsv`. These files can be reproduced from the original sequencing data by running the code in  
[Jeon et.al](https://github.com/xph9876/RNA-mediated_DSB_repair/) (see the folder `NHEJ/data_3_window`).

We process this data as follows:
1. For each table, we isolate the DNA subsequences inserted at the repair site.
2. For individual replicates, we discard sequences with frequency below $10^{-5}$.
3. To compute an average over the four replicates for each experimental design, we:
   - retain only sequences appearing in all four replicates, and
   - retain only sequences with **average frequency** of at least $10^{-5}$.

The resulting processed datasets are saved as `.pickle` files under `data/preprocessed` for efficient use in the next steps.


In [1]:
import os
import pandas as pd
import numpy as np
import pickle as pkl

from utils.preprocess import format_experiment_table

In [2]:
file_root='data/raw/'

#Folders with the data
Human_Files=["KO_sgAB_R1_branch",
"KO_sgAB_R1_cmv",
"KO_sgAB_R1_sense",

"KO_sgAB_R2_branch",
"KO_sgAB_R2_cmv",
"KO_sgAB_R2_sense",

"KO_sgA_R1_branch",
"KO_sgA_R1_cmv",
"KO_sgA_R1_sense",

"KO_sgB_R2_branch",
"KO_sgB_R2_cmv",
"KO_sgB_R2_sense",

"WT_sgAB_R1_branch",
"WT_sgAB_R1_cmv",
"WT_sgAB_R1_sense",

"WT_sgAB_R2_branch",
"WT_sgAB_R2_cmv",
"WT_sgAB_R2_sense",

"WT_sgA_R1_branch",
"WT_sgA_R1_cmv",
"WT_sgA_R1_sense",

"WT_sgB_R2_branch",
"WT_sgB_R2_cmv",
"WT_sgB_R2_sense",

"WT_sgCD_R1_antisense_merged",
"WT_sgCD_R1_splicing_merged",

"WT_sgCD_R2_antisense_merged",
"WT_sgCD_R2_splicing_merged"]

Yeast_Files=[ "YRNS3_YsgCD_R1_antisense",
"YRNS3_YsgCD_R1_branch",

"YRNS3_YsgCD_R2_antisense",
"YRNS3_YsgCD_R2_branch",

"YWTS3_YsgCD_R1_antisense",
"YWTS3_YsgCD_R1_branch",

"YWTS3_YsgCD_R2_antisense",
"YWTS3_YsgCD_R2_branch",

"YWT_YsgCD_R1_antisense",
"YWT_YsgCD_R1_branch",

"YWT_YsgCD_R2_antisense",
"YWT_YsgCD_R2_branch"]

In [3]:
def extract_tables(Files, min_threshold):
    experiments_all=[]
    experiments_average=[]

    exp_id=0
    for i in range(len(Files)):
        experimental_design_name=Files[i] 
        file_name=experimental_design_name+"/window_freq_withoutSubst.tsv"
        
        #reformat name
        experimental_design_name='_'.join(experimental_design_name.split('_')[:4])
        exp_id+=1
        
        Experiment_Table=format_experiment_table(file_root, file_name)
        Libraries=Experiment_Table.library.unique()
        
        #Individual Libraries
        for lib in Libraries:
            Replicate=Experiment_Table[Experiment_Table['library'] == lib].drop(columns='library')
            
            #Use Threshold
            Replicate=Replicate[Replicate['freq']>=min_threshold]
                    
        
            #Save replicate
            replicate_name=experimental_design_name+'_'+lib
            experiments_all.append({'exp_name':replicate_name, 'table':Replicate.copy()})
        
        #Combined Table
        Combined = Experiment_Table.\
            pivot_table(index='sequence', columns='library', values='freq', fill_value=0).\
                reset_index()
        
        # Compute min and average across the library columns
        library_cols = Combined.columns.drop('sequence')  # all library columns
        Combined['min_freq'] = Combined[library_cols].min(axis=1)
        Combined['avg_freq'] = Combined[library_cols].mean(axis=1)
        
        #Apply Threshold
        #Only consider cases where the word appears in four replicates
        Combined=Combined[Combined['min_freq']>0]
        Combined=Combined[Combined['avg_freq']>=min_threshold]
        
        #Save table
        experiments_average.append({'exp_name':experimental_design_name, 'table':Combined.copy()})

    return experiments_all, experiments_average

In [4]:
#Format Human Files
min_threshold=0
Human_experiments_all,Human_experiments_average=extract_tables(Human_Files, min_threshold)

#Save Tables with pickle objects
with open('data/preprocessed/Human_experiments_all.pickle', 'wb') as handle:
    pkl.dump(Human_experiments_all, handle)
with open('data/preprocessed/Human_experiments_average.pickle', 'wb') as handle:
    pkl.dump(Human_experiments_average, handle)

In [5]:
#Format Yeast Files
min_threshold=0
Yeast_experiments_all,Yeast_experiments_average=extract_tables(Yeast_Files, min_threshold)

#Save Tables with pickle objects
with open('data/preprocessed/Yeast_experiments_all.pickle', 'wb') as handle:
    pkl.dump(Yeast_experiments_all, handle)
with open('data/preprocessed/Yeast_experiments_average.pickle', 'wb') as handle:
    pkl.dump(Yeast_experiments_average, handle)